<a href="https://colab.research.google.com/github/hatopopvr/MyPPSimulator/blob/main/MyPPSimulator_En.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MyPPSimulator_Beta_En

<b>Data</b>
- Score Data from ScoreSaber Public API - [doc](https://docs.scoresaber.com/)  
- Ranked Map Data from RankedMapData by rakkyo150 - [RankedMapData](https://github.com/rakkyo150/RankedMapData)  
- Curve Data from ScoreSaber Discord Server - [curve-discussion channel](https://discord.com/channels/501624026532151296/1105521706057072811)  


<b>Author</b>
- hatopop ([@hatopop_vr](https://twitter.com/hatopop_vr))

<b>Caution</b>
- If you would like me to add a default value for the timezone, please mentions me on Twitter.

In [1]:
#@title # Input Information ※
#@markdown ---
#@markdown <h4>Input Data</h4>

#@markdown <font size="3">`player_id` : PlayerID for ScoreSaber. <b>Must</b> be changed.<br/></font>  
player_id =  76561198412839195#@param {type:"number"}

#markdown <font size="3">`google_drive_mount_path` ：GoogleDriveのマウント先です。変更不要。</font><br/> 
google_drive_mount_path = "/content/drive" #param {type:"string"}

#@markdown <font size="3">`google_drive_dir_path` : The directory to save data in GoogleDrive. Optional change.</font> 
google_drive_dir_path = "/MyDrive/MyBeatSaberAnalysis/data" #@param {type:"string"}

#@markdown ---
#@markdown <h4>Setting</h4>

#@markdown <font size="3">`saved_player_score_is_enable`: Whether to use saved score data. Download only the difference.</font>
saved_player_score_is_enable = True #@param {type:"boolean"}
#@markdown <font size="3">`acc_recalq_override_is_enable`: Whether to use the recalculated Accuracy value based on the number of Notes and Combo.</font> 
acc_recalq_override_is_enable = True #@param {type:"boolean"}
# #@markdown <font size="3">`beatleader_data_is_enable`: Whether to retrieve and use data from beatleader.</font>
# beatleader_data_is_enable = True #@param {type:"boolean"}
#markdown <font size="3">`ranked_song_form_leaderboard_is_enable`: Ranked譜面クリア進捗用のレベル別譜面数をScoreSaberのLeaderBoardから取得し直すか。☑で使用。<br /></font>
ranked_song_from_leaderboard_is_enable = False 
#markdown <font size="3">`ss_plus_is_enable`: AccRank区分にSS+を使用するか。☑で使用。  </font><br /> 
ss_plus_is_enable = True #param {type:"boolean"}
#markdown <font size="3">`ss_plus_val`:SS+の設定値です。必要に応じて変更ください。範囲は91-99。  
ss_plus_val = 95 #param {type:"slider", min:91, max:99, step:1}
#@markdown <font size="3">`Recent`:The number of days to be treated as the most recent results. 0 is the same date as today in timezone. If 1, from yesterday.
Recent =  1#@param {type:"integer"}

#@markdown <font size="3">`timezone`:Input or Select your timezone.<br />
#@markdown <font size="3"> The following URL can be used as a reference for time zones ([mjrulesamrat/countryinfo.py](https://gist.github.com/mjrulesamrat/0c1f7de951d3c508fb3a20b4b0b33a98))
timezone = "Asia/Tokyo" #@param ["Asia/Tokyo", "US/Pacific", ""] {allow-input: true}
#@markdown <font size="3">`score_mode`:Select the score you want to get, where `HigherScore` is the higher of `BaseScore` and `ModifieredScore`.</font>
score_mode = "ModifieredScore" #@param ["BaseScore", "ModifieredScore", "HigherScore"]

# #@markdown <font size="3">`curve_mode`:Select the pp-curve you want to get, where `HigherScore` is the higher of `BaseScore` and `ModifieredScore`.</font>
# curve_mode = "CurrentCurve" #@param ["CurrentCurve", "V2.9Curve",  "2023Curve"]

# #@markdown <font size="3">`c`:PP-curve parameter. Basically, don't touch it. default value:`42.113`</font>
# c = 42.113 #@param {type:"number"}

#@title Libraryの取得
!pip install GitPython
import warnings
warnings.filterwarnings("ignore")
import os
import shutil
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime, timedelta
from dateutil import tz
import plotly.express as px
from ipywidgets import interact, Select, BoundedIntText, IntSlider, ToggleButtons, Layout, HBox, VBox, AppLayout
import time
from tqdm import tqdm
import requests
import math
import git
import gc
from IPython.display import HTML, Javascript,Image
from google.colab import drive, files
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import copy
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from scipy import interpolate

#@title Timezone UTC->日本時間+9:00での実行日時の取得(tz_ja)
# tz_ja = pd.Timestamp(datetime.now()).tz_localize('UTC').tz_convert('Asia/Tokyo')
tz_ja = pd.Timestamp(datetime.now()).tz_localize('UTC').tz_convert(timezone)
#today_tz_ja = datetime_now_tz_ja.strftime("%Y.%m.%d")
# print("get_date:{} timezone:{}".format(tz_ja, timezone))
#@title Google Driveのマウント
drive.mount(google_drive_mount_path)

#@title その他内部設定値
# データ元のURL
## song_data_zip_git_url: ScoreSaberの全曲情報のzip(json)のURLです。変更なければそのままで。 
song_data_zip_git_url = "https://github.com/andruzzzhka/BeatSaberScrappedData.git"
## ranked_excluded_data_git_url: ScoreSaberのRank譜面除外リスト(csv)のURLです。暫定的処置。 
ranked_excluded_data_git_url = "https://github.com/hatopopvr/ScoreSaberRankedExcludedMaps.git"
## rankedmapdata_url: BeatSaverデータのcsvのURLです。らっきょさんデータ。 
rankedmapdata_url = 'https://api.github.com/repos/rakkyo150/RankedMapData/releases'
# google drive内のdata置き場親フォルダ
data_path = r"{}{}".format(google_drive_mount_path, google_drive_dir_path)
# player情報の親フォルダ(data_pathの子フォルダ)
player_path = r"{}/players_data/{}".format(data_path, player_id)
## playerinfoの保存先
player_info_path = r"{}/player_info_{}.csv".format(player_path, player_id)
## playerのscore関連保存先
player_score_path = r"{}/scores_full_{}.csv".format(player_path, player_id)
player_ranked_path = r"{}/scores_ranked_{}.csv".format(player_path, player_id)
## playerのscore関連保存先
player_score_pickle_path = r"{}/scores_full_{}.pkl".format(player_path, player_id)
player_ranked_pickle_path = r"{}/scores_ranked_{}.pkl".format(player_path, player_id)
## 曲情報の保存関連
song_clone_path = r"/content/BeatSaberScrappedData"
song_zip_path = r"{}/combinedScrappedData.zip".format(song_clone_path)
song_json_path = r"{}/combinedScrappedData.json".format(data_path)
## 曲情報の保存先
song_list_path = r"{}/song_list_full.csv".format(data_path)
song_ranked_path = r"{}/song_ranked.csv".format(data_path)
# ランク除外関連パス
ranked_excluded_clone_dir_path = r"/content/ScoreSaberRankedExcludedMaps"
ranked_excluded_clone_csv_path = r"{}/RankedExcludedMaps.csv".format(ranked_excluded_clone_dir_path)
ranked_excluded_csv_path = r"{}/RankedExcludedMaps.csv".format(data_path)
# levelclearランク除外関連パス
level_cleared_path = r"{}/level_cleared_{}.csv".format(player_path, player_id)

# playlistの保存
song_playlist_path = r"{}/playlists".format(data_path)
song_worst_playlist_path = r"{}/worst_playlist_{}.json".format(song_playlist_path, tz_ja.strftime("%Y%m%d"))
song_top_playlist_path = r"{}/top_playlist_{}.json".format(song_playlist_path, tz_ja.strftime("%Y%m%d"))

# SS_plus設定値
ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)
# その他 colab表示など設定
pd.options.display.precision = 2
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
page_count=100 #ScoreSaberの1ページあたりのページ数

# 出力最大高さ
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

get_ipython().events.register('pre_run_cell', resize_colab_cell)

#@title 列情報の設定
# Player Infoの記録用列 (TotalFC, RankedFCは別途結合)
cols_info =[
    "Pic"
    ,"name"
    ,"country"
    ,"pp"
    ,"rank"
    ,"countryRank"
    ,"role"
    ,"TotalScore"
    ,"RankedScore"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
    ,"ReplayWatched"
    ,"ScoreDate"
    ,"TotalFC"
    ,"RankedFC"
    ,"TotalPlayRank"
    ,"TotalPlayJPRank" 
    ,"RankedPlayRank"
    ,"RankedPlayJPRank"
    ,"TotalScoreRank" 
    ,"TotalScoreJPRank"
    ,"RankedScoreRank"
    ,"RankedScoreJPRank" 
    ,"AveRankedAccRank"
    ,"AveRankedAccJPRank"
]

# Player Infoの表示用列 (TotalFC, RankedFCは別途結合)
cols_info_sort =[
    "Pic"
    ,"name"
    ,"country"
    ,"pp"
    ,"rank"
    ,"countryRank"
    ,"role"
    ,"TotalScore"
    ,"RankedScore"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
    ,"TotalFC"
    ,"RankedFC"
    ,"ReplayWatched"
    ,"ScoreDateTz"
]

# リュナンさんのScoreSaberRanking表示用列(不使用)
cols_info_rank = [
    "TotalPlayRank"
    ,"TotalPlayJPRank" 
    ,"RankedPlayRank"
    ,"RankedPlayJPRank"
    ,"TotalScoreRank" 
    ,"TotalScoreJPRank"
    ,"RankedScoreRank"
    ,"RankedScoreJPRank" 
    ,"AveRankedAccRank"
    ,"AveRankedAccJPRank"
]

# summary用列
cols_summary =[
    "rank"
    ,"countryRank"
    ,"pp"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
]

# SongList抜粋用列
cols_song =[
    "Cover"
    ,"Song"
    ,"Level"
    ,"LevelStr"
    ,"Stars"
    # ,"maxPP"
    # ,"maxScore"
    ,"Difficulty"
    # ,"Play"
    # ,"DailyPlay"
    ,"CreatedDateJa"
    ,"RankDateJa"
    ,"Bombs"
    ,"Notes"
    ,"Obstacles"
    ,"Njs"
    ,"NjsOffset"
    ,"Bpm"
    ,"Upvotes"
    #,"Downvotes"
    ,"Duration"
    ,"Ranked"
]

# Rank譜面除外リスト結合用列
cols_excluded = [
                 'Hash',
                 'Difficulty',
                 'RankedExcluded'
                 ]

# Score用列
cols_score =[
    "Cover"
    ,"Song"
    ,"Level"
    ,"Stars"
    ,"Acc"
    ,"AccRank"
    ,"FC"
    ,"Rank"
    ,"PP"
    ,"Miss"
    ,"Bad"
    ,"Combo"
    ,"Score"
    #,"Mode"
    ,"Difficulty"
    ,"Play"
    ,"DailyPlay"
    ,"Bpm"
    ,"Duration"
    ,"Notes"
    ,'Nps'
    ,"Njs"
    ,"Bombs"
    ,"Obstacles"
    # ,"NjsOffset"
    ,'Upvotesratio' 
    ,"Upvotes"
    ,"Downvotes"
    ,"Ranked"
    ,"Days"
    ,"Months"
    ,"Tags"
    ,"Preview"
 ]

# Acc再計算用列
cols_recalq = [
    'Hash',
    'Cover',
    'SongName',
    'Difficulty', 'Stars', 
    # 'Ranked', 
    'Notes', 
    'Acc', 
    'AccRecalq', 
    'AccDiff',
    'AccRank', 
    'AccRankRecalq', 
    'MaxScore',
    'MaxScoreRecalq',
    'MaxScoreDiff',
    'Score', 
    'Miss', 
    'Combo',
    # 'RankDateJa',
    # 'CreatedDateJa', 
    'Preview'
]

# ScatterPlot用X-Y軸列
cols_xy = [
    "Stars"
    ,"Level"
    ,"Acc"
    ,"Rank"
    ,"PP"
    ,"Miss"
    ,"Bad"
    ,"Combo"
    ,"Score"
    ,"Play"
    ,"DailyPlay"
    ,"Days"
    ,"Months"
    ,"Bpm"
    ,"Duration"
    ,"Notes"
    ,'Nps'
    ,"Njs"
    ,"Bombs"
    ,"Obstacles"
    ,"NjsOffset"
    ,'Upvotesratio'
    ,"Upvotes"
    ,"Downvotes"
]

# RankedMap(BeatSaver)結合用列
cols_rankedmap = ['Hash', 
                  'Difficulty', 
                  'Upvotesratio',
                  'Nps', 
                  'Tags', 'Preview'
]

# ScatterPlot用カテゴリ
cols_cate = [
    "LevelStr"
    ,"Days"
    ,"DaysStr"
    ,"Months"
    ,"MonthsStr"
    ,"RecentStr"
    ,"Acc"
    ,"AccRank"
    ,"FC"
    ,"SongAuthor"
    ,"LevelAuthor"
]

# Worst-Top ScatterPlot用カテゴリ
cols_cate_wt = [
    "WorstTop"
    ,"LevelStr"
    ,"Days"
    ,"DaysStr"
    ,"Months"
    ,"MonthsStr"
    ,"RecentStr"
    ,"Acc"
    ,"AccRank"
    ,"FC"
    ,"SongAuthor"
    ,"LevelAuthor"
]

# Recent_history用の列
cols_Recent_history = ['Cover', 'Song', 'Difficulty', 'Stars', 'Acc', 'AccRank', 'FC', 'Rank', 'PP',
       'Miss', 'Bad', 'Combo', 'Score']


cols_worst_n = ['Cover', 'Song', 'Difficulty', 'Level', 'Stars', 'Acc', 'AccRank', 'FC', 'Rank', 'PP',
       'Miss', 'Bad', 'Combo', 'Score']

#@title スタイルの設定

cover_image_size=70 #カバー,プロファイル画像のサイズ単位px
info_image_size=150 #Player画像のサイズ単位px

style_format = {
        "rank": "#{:,.0f}",
        "countryRank": "#{:,.0f}",
        "TotalScore": "{:,.0f}",
        "RankedScore": "{:,.0f}",
        "AveRankedAcc": "{:,.2f}%",
        "TotalPlay": "{:,.0f}",
        "RankedPlay": "{:,.0f}",
        "TotalFC": "{:,.0f}",
        "RankedFC": "{:,.0f}",
        "ReplayWatched": "{:,.0f}",
        "Level": "{:,.0f}",
        "Stars": "{:,.2f}★",
        "Acc": "{:,.2f}%",
        "AccRecalq": "{:,.2f}%",
        "AccDiff": "{:,.2f}%",
        "MaxScore": "{:,.0f}",
        "MaxScoreRecalq": "{:,.0f}",
        "MaxScoreDiff": "{:,.0f}",
        "Score": "{:,.0f}",
        "Rank": "#{:,.0f}",
        "pp": "{:,.2f}pp",
        "PP": "{:,.2f}pp",
        "Miss": "{:,.0f}",
        "Bad": "{:,.0f}",
        "Combo": "{:,.0f}",
        "Score": "{:,.0f}",
        "Play": "{:,.0f}",
        "DailyPlay": "{:,.0f}",
        "Bpm": "{:,.0f}",
        "Duration": "{:,.0f}",
        "Notes": "{:,.0f}",
        "Nps": "{:,.2f}",
        "Njs": "{:,.0f}",
        "Bombs": "{:,.0f}",
        "Obstacles": "{:,.0f}",
        "NjsOffset": "{:,.2f}",
        "Upvotesratio": "{:,.2f}",
        "Upvotes": "{:,.0f}",
        "Downvotes": "{:,.0f}",
        "Days": "{:,.0f}",
        "Months": "{:,.0f}",
        "Song": "{:,.0f}",
        "RecentCleared": "{:,.0f}",
        "Cleared": "{:,.0f}",
        "NF": "{:,.0f}",
        "NotCleared": "{:,.0f}",
        "AlreadyCleared": "{:,.0f}",
        "NotClearedRate": "{:,.2f}%",
        "AlreadyClearedRate": "{:,.2f}%",
        "RecentClearedRate": "{:,.2f}%",
        "NFRate": "{:,.2f}%",
        "SSS": "{:,.0f}",
        ss_plus: "{:,.0f}",
        "SS": "{:,.0f}",
        "S": "{:,.0f}",
        "A": "{:,.0f}",
        "B": "{:,.0f}",
        "Other": "{:,.0f}",
        "SSS-Rate": "{:,.2f}%",
        "SS-Rate": "{:,.2f}%",
        ss_plus_rate: "{:,.2f}%",
        "S-Rate": "{:,.2f}%",
        "A-Rate": "{:,.2f}%",
        "B-Rate": "{:,.2f}%",
        "Other-Rate": "{:,.2f}%",
        "FC": "{:,.0f}",
        "RecentFC": "{:,.0f}",
        "AlreadyFC": "{:,.0f}",
        "NotFC": "{:,.0f}",
        "AlreadyFCRate": "{:,.2f}%",
        "RecentFCRate": "{:,.2f}%",
        "NotFCRate": "{:,.2f}%",
        "Song": "{:,.0f}",
        "TotalPlayRank": "#{:,.0f}",
        "TotalPlayJPRank": "#{:,.0f}",
        "RankedPlayRank": "#{:,.0f}",
        "RankedPlayJPRank": "#{:,.0f}",
        "TotalScoreRank": "#{:,.0f}",
        "TotalScoreJPRank": "#{:,.0f}",
        "RankedScoreRank": "#{:,.0f}",
        "RankedScoreJPRank": "#{:,.0f}",
        "AveRankedAccRank": "#{:,.0f}",
        "AveRankedAccJPRank": "#{:,.0f}",
     }

ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)

style_worst_top = {
        "Level": "{:,.0f}",
        "Stars": "{:,.2f}★",
        "Acc": "{:,.2f}%",
        "Rank": "#{:,.0f}",
        "pp": "{:,.2f}pp",
        "PP": "{:,.2f}pp",
        "Miss": "{:,.0f}",
        "Bad": "{:,.0f}",
        "Combo": "{:,.0f}",
        "Score": "{:,.0f}",
        "Play": "{:,.0f}",
        "DailyPlay": "{:,.0f}",
        "Bpm": "{:,.0f}",
        "Duration": "{:,.0f}",
        "Notes": "{:,.0f}",
        "Nps": "{:,.2f}",
        "Njs": "{:,.0f}",
        "Bombs": "{:,.0f}",
        "Obstacles": "{:,.0f}",
        "NjsOffset": "{:,.2f}",
        "Upvotesratio": "{:,.2f}",
        "Upvotes": "{:,.0f}",
        "Downvotes": "{:,.0f}",
        "Days": "{:,.0f}",
        "Months": "{:,.0f}",
     }

def color_negative_red(val):
    color = 'red' if val < 0 else 'blue'
    return 'color: %s' % color

# Difficulty color
def color_difficulty(val):
    if val == "Easy":
        return 'color: #66BB6A; font-weight: bold;'
        color = "#66BB6A"
    elif val == "Normal":
        return 'color: #29B6F6; font-weight: bold;'
        color = "#29B6F6"
    elif val == "Hard":
        return 'color: #FB8C00; font-weight: bold;'
        color = "#FB8C00"
    elif val == "Expert":
        return 'color: #E53935; font-weight: bold;'
        color = "#E53935"
    elif val == "Expert+":
        return 'color: #8E24AA; font-weight: bold;'
        color = "#8E24AA"
    elif val == "ExpertPlus":
        return 'color: #8E24AA; font-weight: bold;'
        color = "#8E24AA"
    return 'color: %s' % color

# Acc Rank color
def color_acc_rank(val):
    if val == "SSS":
        return 'color: #00ffff; font-weight: bold;'
    elif ss_plus_is_enable and val == ss_plus:
        return 'color: #636EFA; font-weight: bold;'
    elif val == "SS":
        return 'color: #ff4500; font-weight: bold;'
    elif val == "S":
        return 'color: #ffaaff; font-weight: bold;'
    elif val == "A":
        return 'color: #ffcc66; font-weight: bold;'
        #color = "#eebb55"
    elif val == "B":
        return 'color: #cccc66; font-weight: bold;'
    else:
        return 'color: #666666; font-weight: bold;'
    # return 'color: %s' % color


# Acc Rank color
def color_acc_player(val):
    if val == 100:
        return 'color: #00ffff; font-weight: bold;'
    elif ss_plus_is_enable and val >= ss_plus_val:
        return 'color: #636EFA; font-weight: bold;'
    elif val >= 90:
        return 'color: #ff4500; font-weight: bold;'
    elif val == 80:
        return 'color: #ffaaff; font-weight: bold;'
    elif val == 65:
        return 'color: #ffcc66; font-weight: bold;'
        #color = "#eebb55"
    elif val == 50:
        return 'color: #cccc66; font-weight: bold;'
    else:
        return 'color: #666666; font-weight: bold;'
    # return 'color: %s' % color


# FC color
def color_fc(val):
    if val == "FC":
        color = "#23D160"
    else:
        color = "black"        
    return 'color: %s' % color

def color_fc(val):
    if val == "FC":
        return 'color: #23D160; font-weight: bold;'
    else:
        return 'color: black'


# Player Info Table Format
styles_info = [
    dict(selector="td", props=[("font-size", "130%"),
                               ("text-align", "center"),
                               ("padding-top", "0px"),
                               ("padding-bottom", "0px"),                              
                               ])
]

# Player Info Table Format
styles_data = [
    {"selector":"td", "props":[("padding-top", "0px"),
                               ("padding-bottom", "0px"),                              
                               ]},
    {"selector":"th", "props":[("padding-top", "0px"),
                               ("padding-bottom", "0px"),                              
                               ]},
]

# Caption Table Format
caption_styles = [dict(selector="caption",
                       props=[
                              ("text-align", "lett"),
                              ("font-size", "150%"),
                              ("border-bottom", "1px solid"),
                              ]
                       )
                  ]



# タイムゾーンの設定-------------------------


#@title 各種フォルダの作成
# 各種パスのディレクトリ作成
## データ大元のフォルダ作成
if os.path.exists(data_path) == False:
    print('MyBeatSaberAnalytics用のデータ格納フォルダをGoogle Driveに新規作成します。')
    print('データ格納フォルダ:{}'.format(data_path))     
    os.makedirs(data_path, exist_ok=True)
    print('作成が完了しました。')         

## データ大元のフォルダ作成
if os.path.exists(player_path) == False:
    old_player_info_path = r"{}/player_info_{}.csv".format(data_path, player_id)
    print('PlayerID:{}用のデータ格納フォルダを新規作成します。'.format(player_id))
    print('playerフォルダ:{}'.format(player_path))     
    print('作成が完了しました。')         
    os.makedirs(player_path, exist_ok=True)
    if os.path.isfile(old_player_info_path):
        print('データ引継ぎのため、player_infoｂのcsvファイルを上記フォルダに移動させます。')
        print('移動前:{}'.format(old_player_info_path))
        print('移動後:{}'.format(player_info_path))
        shutil.move(old_player_info_path, player_info_path)
        print('移動が完了しました。')

## プレイヤー用のフォルダ作成
if os.path.exists(song_playlist_path) == False:
    print('Playlist格納用のフォルダを新規作成します。')
    print('Playlist格納フォルダ:{}'.format(song_playlist_path))     
    os.makedirs(song_playlist_path, exist_ok=True)
    print('作成が完了しました。')

#@title Get Player Info (df_info)
url = r"https://scoresaber.com/api/player/{}/full".format(player_id)
response = requests.get(url)
res_data = response.json()
df_info = json_normalize(res_data)

#df_info["Pic"] = '<img src="'+df_info["profilePicture"]+'"/>'
df_info["Pic"] = '<img src="'+df_info["profilePicture"]+'" style="width:{}px;"/>'.format(info_image_size)
df_info["TotalScore"] = df_info["scoreStats.totalScore"]
df_info["RankedScore"] = df_info["scoreStats.totalRankedScore"]
df_info["AveRankedAcc"] = df_info["scoreStats.averageRankedAccuracy"]
df_info["TotalPlay"] = df_info["scoreStats.totalPlayCount"]
df_info["RankedPlay"] = df_info["scoreStats.rankedPlayCount"]
df_info["ReplayWatched"] = df_info["scoreStats.replaysWatched"]
df_info["ScoreDate"] = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

df_info["ScoreDateUtc"] = pd.to_datetime(df_info['ScoreDate'], utc=True)
_df_info_idx = df_info.set_index("ScoreDateUtc")
df_info["ScoreDateTz"] = _df_info_idx.index.tz_convert(timezone)

PlayCount = df_info["TotalPlay"][0]
RankedPlay = df_info["RankedPlay"][0]
RangeCount = math.ceil(PlayCount / page_count) + 1

print("------------------------------------")
print("timezone:{} | datetime:{} ".format(timezone, tz_ja))
# print("Player:{}, TotalPlayCount:{:,}, RankedPlayCount:{:,}, PageCount:{}".format(df_info["name"][0], PlayCount, RankedPlay, RangeCount))

# display(df_info[['Pic','name','TotalPlay']].style.set_table_styles(styles_info).format(style_format, na_rep="-"))

# ----------------------------------------
# Display Player Information
# ----------------------------------------
def str_difficulty(val):
    if val == "Easy":
        return 'E'
    elif val == "Normal":
        return 'N'
    elif val == "Hard":
        return 'H'
    elif val == "Expert":
        return 'Ex'
    elif val == "Expert+":
        return 'Ex+'
    elif val == "ExpertPlus":
        return 'Ex+'
    return '-'

def str_fc(val):
    if val == "FC":
        return 'FC'
    else:
        return ''

# FC color
def color_fc(val):
    if val == "FC":
        color = "#23D160"
    else:
        color = "black"        
    return 'color: %s' % color

# pp color
def color_pp(val):
    if val >= target_pp:
        color = "red"
    else:
        color = "black"        
    return 'color: %s' % color

# rank color
def color_rank(val):
    if val <= target_global_rank:
        color = "red"
    else:
        color = "black"        
    return 'color: %s' % color

def color_latest(x):
    if  x == 1:
        color = "red"
    else:
        color = "black"        
    return 'color: %s' % color

# weight
def format_weight(val):
    if val <= target_weight:
        return "".format(val)
    else:
        return "({:.1f}pp)".format(val)

def format_star(val):
    if val > 0:
        return "{:.2f}★".format(val)
    else:
        return ""
    return "{:.2f}★".format(val)        

def format_acc(val):
    if val <= 100:
        return "{:.2f}%".format(val)
    else:
        return ""
    return "{:.2f}%".format(val)

def format_pp(val):
    if val > 0:
        return "{:.1f}pp".format(val)
    else:
        return ""
    return "{:.1f}pp".format(val)

def func_unrank(x):
    if x == 0:
        return None
    else:
        return x

def func_tile_css(_width=1000,
             _cover_image_size=200,
             _star_font_size=22,
             _diff_font_size=18,
             _acc_font_size=18,
             _fc_font_size=18,
             _pp_font_size=22,
             _weight_font_size=8,
             _rank_font_size=22,
             _date_font_size=18,
             _player_font_size=22,
             _notes_acc_font_size=24
             ):
    return """
        <style>
        .flex_box {
            width:""" + str(_width) + """px;
            display: flex; /* フレックスボックスにする */;
            flex-wrap: wrap;
            padding:0px;
            margin:0px;
        }
        .flex_item {
            display: inline-block;
            position: relative;
            padding:0px 5px 0px 0px;
            margin:0px;
        }
        .flex_str_top, .flex_str_bottom_left, .flex_str_bottom_right{
            position: absolute;
            font-weight: bold;
            white-space:nowrap;
            overflow:hidden;
            color: #000;
            width:""" + str(_cover_image_size-10) + """px;
            text-shadow:1px 1px 0 #FFF, -1px -1px 0 #FFF,
                    -1px 1px 0 #FFF, 1px -1px 0 #FFF,
                    0px 1px 0 #FFF,  0-1px 0 #FFF,
                    -1px 0 0 #FFF, 1px 0 0 #FFF;
        }

        .flex_str_top {
            left: 5px;
            top: 5px;
        }

        .flex_str_bottom_left {
            left: 5px;
            bottom: 5px;
        }

        .flex_str_bottom_right {
            left: 5px;
            bottom: 5px;
        }

        .flex_star, .flex_acc, .flex_accrank, .flex_fc ,.flex_pp, .flex_pp_player, .flex_weight, .flex_rank {
            padding:0px;
            margin:0px;
        }

        .flex_star{
            font-size: """ + str(_star_font_size) + """px;
        }

        .flex_diff{
            font-size: """ + str(_diff_font_size) + """px;
        }

        .flex_acc{
            font-size: """ + str(_acc_font_size) + """px;
        }

        .flex_fc {
            font-size: """ + str(_fc_font_size) + """px;
        }

        .flex_pp{
            font-size: """ + str(_pp_font_size) + """px;
        }

        .flex_pp_player{
            font-size: """ + str(_pp_font_size-2) + """px;
        }

        .flex_rank, .flex_rank_player, .flex_modifier{
            padding:0px;
            margin:0px;
            text-align:right;
        }

        .flex_rank{
            font-size: """ + str(_rank_font_size) + """px;
        }

        .flex_rank_player {
            font-size: """ + str(_rank_font_size-2) + """px;
        }

        .flex_player{
            font-size: """ + str(_player_font_size) + """px;
        }

        .flex_weight{
            font-size: """ + str(_weight_font_size) + """px;
        }


        .flex_date {
            font-size: """ + str(_date_font_size) + """px;
            text-align:right;
        }

        .flex_modifier{
            color: #666;
            font-size: """ + str(_fc_font_size) + """px;
        }


        .flex_str_center{
            position: absolute;
            font-weight: bold;
            white-space:nowrap;
            overflow:hidden;
            color: #000;
            width:""" + str(_cover_image_size) + """px;
            text-shadow:1px 1px 0 #FFF, -1px -1px 0 #FFF,
                    -1px 1px 0 #FFF, 1px -1px 0 #FFF,
                    0px 1px 0 #FFF,  0-1px 0 #FFF,
                    -1px 0 0 #FFF, 1px 0 0 #FFF;
        }

        .flex_str_center {
            top:""" + str(_cover_image_size / 2 -10) + """px;     
            text-align:left;
            justify-content: center;
       }

        .flex_notesacc {
            display: flex;
            justify-content: space-between;
            align-items: center;
            flex-wrap: wrap;
         }

        .flex_acc_left, .flex_acc_right {
            display: flex;
            margin-right: 5px;
            margin-left: 5px;
            font-size: """ + str(_notes_acc_font_size) + """px;
            font-weight: 700;
        }

        .flex_acc_left {
            background:linear-gradient(transparent 92.5%, #EE6666 10%);
        }

        .flex_acc_right {
            background:linear-gradient(transparent 92.5%, #6666EE 10%);
        }

        h1 {
            padding: 3px 0 3px 3px;
            width: 100%;
            font-size: 150%;
            border-left: 4px solid #FFDE18;
        }

        </style>
        """


# ----------------------------------------
# Player html 
# ----------------------------------------
def func_player(_cover_image_size=200, _fc_is_enable=False):
    div_img = "<a href='https://scoresaber.com/u/{}' target='_blank'><img src='{}' style='width:{}px; height:{}px'/></a>".format(player_id, df_info["profilePicture"][0],_cover_image_size, _cover_image_size)
    div_name = "<div class='flex_player'>{}</div>".format(df_info["name"][0])

    div_playcount = "<div class='flex_song'>TotalPlayCount:{:,.0f}</div>".format(df_info["TotalPlay"][0])
    div_rankedplaycount = "<div class='flex_song'>RankedPlayCount:{:,.0f}</div>".format(df_info["RankedPlay"][0])
    div_playcount = "<div class='flex_song'>RankedPlay:{:,.0f}(Total:{:,.0f})</div>".format(df_info["RankedPlay"][0],df_info["TotalPlay"][0])
    div_fccount = "<div class='flex_song'>RankedFC:{:,.0f}(Total:{:,.0f})</div>".format(df_info["RankedFC"][0],df_info["TotalFC"][0]) if _fc_is_enable else ""
    span_acc = "<span class='flex_acc' style='{}'>{}</span>".format(color_acc_player(df_info["AveRankedAcc"][0]), format_acc(df_info["AveRankedAcc"][0]))
    div_acc = "<div>{}</div>".format(span_acc)

    span_pp = "<span class='flex_pp_player'>{}</span>".format((format_pp(df_info["pp"][0])))
    div_pp = "<div>{}</div>".format(span_pp)
    div_country_rank = "<div class='flex_rank_player'>#{:,.0f}[{}]</div>".format(df_info["countryRank"][0], df_info["country"][0])
    div_rank = "<div class='flex_rank_player'>#{:,.0f}</div>".format(df_info["rank"][0])
    return """
            <div class='flex_item'>{}<br/>
                <div class='flex_str_top'>
                    {}
                    {}
                    {}
                </div>
                <div class='flex_str_bottom_left'>
                    {}
                    {}
                </div>
                <div class='flex_str_bottom_right'>
                    {}
                    {}
                </div>
            </div>
            """.format(div_img, div_name, div_playcount,div_fccount,div_acc, div_pp, div_country_rank, div_rank)


flex_head = """
            <div id="imageDIV" 
                class='flex_box_wrapper'>
                <div class='flex_box'>
            """.format(tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])

flex_player = func_player(200)

flex_end = "</div></div>"

img_css = func_tile_css()

html_str = flex_head + flex_player + flex_end + img_css

HTML(html_str)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
------------------------------------
timezone:Asia/Tokyo | datetime:2023-05-22 14:41:06.567311+09:00 


# Get Data

In [2]:
#@title Get Ranked Song Data
#@markdown ScoreSaber's Ranked Song Data from [RankedMapData](https://github.com/rakkyo150/RankedMapData) by [rakkyo150](https://twitter.com/rakkyo150)

headers = {
    'Accept': 'application/vnd.github.v3+json',
}

response = requests.get(rankedmapdata_url, headers=headers)

data = response.json()

# 最新releaseのcsvのurl取得
url_rankmap_data = data[0]["assets"][0]["browser_download_url"]

file_name = os.path.join(data_path, os.path.basename(url_rankmap_data))
result = requests.get(url_rankmap_data, stream=True)
if result.status_code == 200:
    with open(file_name, 'wb') as file:
        result.raw.decode_content = True
        shutil.copyfileobj(result.raw, file)

df_rankmap_data = pd.read_csv(file_name)
df_rankmap_data = df_rankmap_data[[x for x in df_rankmap_data.columns if not x.startswith("Unnamed")]]
df_rankmap_data["hash"] = df_rankmap_data["hash"].str.upper()
df_rankmap_data["Preview"] = '<audio src="' + df_rankmap_data["previewUrl"] + '" preload="none" controls></audio>'

df_rankmap_data = df_rankmap_data.rename(columns=lambda x: x.capitalize())

df_rankmap_data.rename(columns={
                        "Songname":"SongName",
                        "Songsubname":"SongSub",
                        "Songauthorname":"SongAuthor",
                        "Levelauthorname":"LevelAuthor"
                            }, inplace=True)

# 準備
df_rankmap_data["Song"] = df_rankmap_data["SongName"] + " / " + df_rankmap_data["SongAuthor"] + " [" + df_rankmap_data["LevelAuthor"] + "]"
df_rankmap_data["Level"] = df_rankmap_data["Stars"].astype("int")
df_rankmap_data["LevelStr"] = df_rankmap_data["Level"].astype("str")

print("path:{},count:{:,}".format(file_name,len(df_rankmap_data["Hash"])))

<IPython.core.display.Javascript object>

path:/content/drive/MyDrive/MyBeatSaberAnalysis/data/outcome.csv,count:3,855


In [3]:
#@title Get Player Scores Data
#@markdown Players' Score Data from ScoreSaber Public API - [doc](https://docs.scoresaber.com/)  

def func_mode(x):
    if  x == "SoloStandard":
        return "Standard"
    else:
        return x

def func_score(x):
    if  x > 100:
        return "-"
    elif x == 100:
        return "SSS"
    elif x >= ss_plus_val and ss_plus_is_enable:
        return ss_plus
    elif x >= 90:
        return "SS"
    elif x >= 80:
        return "S"
    elif x >= 65:
        return "A"
    elif x >= 50:
        return "B"
    elif x >= 35:
        return "C"
    elif x >= 20:
        return "D"
    elif x >= 0:
        return "E"
    else:
        return "-"

def func_fc(x):
    if  x:
        return "FC"
    else:
        return "-"

def func_Recent(x):
    if  x <= Recent:
        return 1
    else:
        return 0

# url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&limit={}".format(player_id, page_count)
# response = requests.get(url)
# res_data = response.json()
# df_scores = json_normalize(res_data['playerScores'])

# for i in tqdm(range(2, RangeCount)):
#     url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}&limit={}".format(player_id, i, page_count)
#     try:
#         response = requests.get(url)
#         res_data = response.json()
#         df_scores=df_scores.append(json_normalize(res_data['playerScores']), ignore_index=True)
#     except:
#         break

if saved_player_score_is_enable and os.path.exists(player_score_pickle_path):    
    df_scores_pkl = pd.read_pickle(player_score_pickle_path)
    _df_scores_pkl = df_scores_pkl.head(0)

    i = 1
    while True:
        url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}&limit={}".format(player_id, i, page_count)
        try:
            response = requests.get(url)
            res_data = response.json()
            _df_scores_pkl=_df_scores_pkl.append(json_normalize(res_data['playerScores']), ignore_index=True)
            if df_scores_pkl['score.timeSet'].max() > _df_scores_pkl['score.timeSet'].min():
                break
        except:
            break

    df_scores=df_scores_pkl.append(_df_scores_pkl, ignore_index=True).sort_values("score.timeSet", ascending=False).groupby("score.id").head(1)

else:
    url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&limit={}".format(player_id, page_count)
    response = requests.get(url)
    res_data = response.json()
    df_scores = json_normalize(res_data['playerScores'])

    for i in tqdm(range(2, RangeCount)):
        url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}&limit={}".format(player_id, i, page_count)
        try:
            response = requests.get(url)
            res_data = response.json()
            df_scores=df_scores.append(json_normalize(res_data['playerScores']), ignore_index=True)
        except:
            break

# 未加工データ保存
df_scores.to_pickle(player_score_pickle_path)

# 加工
df_scores['Song'] = df_scores['leaderboard.songName'] + " " + df_scores['leaderboard.songSubName'] + " / " + df_scores['leaderboard.songAuthorName'] + " [" + df_scores['leaderboard.levelAuthorName'] + "]"
df_scores['SongName'] = df_scores['leaderboard.songName']
df_scores['SongSub'] = df_scores['leaderboard.songSubName']
df_scores['SongAuthor'] = df_scores['leaderboard.songAuthorName']
df_scores['LevelAuthor'] = df_scores['leaderboard.levelAuthorName']
df_scores['Hash'] = df_scores['leaderboard.songHash'].str.upper()

def func_higher_score(x):
    try:
        if x[0] >= x[1]:
            return x[0]
        else:
            return x[1]
    except:
        return x[1]

if score_mode == "BaseScore":
    df_scores['Score'] = df_scores['score.baseScore']
elif score_mode == "ModifiedScore":
    df_scores['Score'] = df_scores['score.modifiedScore']
elif score_mode == "HigherScore":
    df_scores['Score'] = df_scores[['score.baseScore','score.modifiedScore']].apply(func_higher_score, axis=1)
else:
    df_scores['Score'] = df_scores['score.modifiedScore']

df_scores['Acc'] = df_scores['Score'] / df_scores['leaderboard.maxScore'] * 100
# df_scores['Acc'] = df_scores['score.modifiedScore'] / df_scores['leaderboard.maxScore'] * 100
df_scores['MaxScore'] = df_scores['leaderboard.maxScore']

df_scores['Mode'] = df_scores['leaderboard.difficulty.gameMode'].apply(func_mode)
#df_scores['Mode'] = df_scores['leaderboard.difficulty.gameMode']
_df_scores = df_scores['leaderboard.difficulty.difficultyRaw'].str.split('_', expand=True)
_df_scores.columns = ['_','Difficulty', 'Mode']
df_scores['Difficulty'] = _df_scores['Difficulty']
df_scores['Stars'] = df_scores['leaderboard.stars']
df_scores['Level'] = df_scores['Stars'].astype('int')
df_scores["LevelStr"] = df_scores['Level'].astype('str')
# df_scores['Score'] = df_scores['score.modifiedScore']
df_scores['Bad'] = df_scores['score.badCuts']
df_scores['Miss'] = df_scores['score.missedNotes']
df_scores['Combo'] = df_scores['score.maxCombo']
df_scores['PP'] = df_scores['score.pp']
df_scores['PPWeight'] = df_scores['score.pp'] * df_scores['score.weight']
df_scores['Rank'] = df_scores['score.rank']
df_scores['Modifiers'] = df_scores['score.modifiers']
df_scores['Ranked'] = df_scores['leaderboard.ranked']
df_scores['Qualified'] = df_scores['leaderboard.qualified']
df_scores['Play'] = df_scores['leaderboard.plays']
df_scores['DailyPlay'] = df_scores['leaderboard.dailyPlays']
df_scores['DateUtc'] = pd.to_datetime(df_scores['score.timeSet'])
_df_scores_idx = df_scores.set_index('DateUtc')
df_scores['DateJa'] = _df_scores_idx.index.tz_convert(timezone)
df_scores['Date'] = df_scores['DateJa'].dt.date
df_scores['Days'] = (tz_ja.date() - df_scores['Date']).dt.days
df_scores = df_scores.set_index('DateJa')
df_scores['Months'] = (df_scores['Days'] / 30).astype('int')
df_scores['DaysStr'] = df_scores['Days'].astype('str')
df_scores['MonthsStr'] = df_scores['Months'].astype('str')

df_scores['Recent'] = df_scores['Days'].apply(func_Recent)
df_scores['RecentStr'] = df_scores['Recent'].astype('str')

df_scores['AccRank'] = df_scores['Acc'].apply(func_score)

df_scores['Cover'] = '<img src="'+df_scores['leaderboard.coverImage']+'" style="width:{}px;"/>'.format(cover_image_size)

df_scores['FC'] = df_scores['score.fullCombo'].apply(func_fc)

df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("score.")]]
df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("leaderboard.")]]


# 結合前Score情報の抽出(LevelClearedProgress用)
df_scores_org = df_scores.copy()

# 改行コード等の除去
for col in df_scores.columns:
    try:
        if len(df_scores[df_scores[col].str.contains("\n")][[col]]) == 0:
            continue
        else:
            df_scores[col] = df_scores[col].str.replace("\n","")
    except:
        continue

for col in df_scores.columns:
    try:
        if len(df_scores[df_scores[col].str.contains("\r")][[col]]) == 0:
            continue
        else:
            df_scores[col] = df_scores[col].str.replace("\r","")
    except:
        continue


# RankedMap(らっきょさんのBeatSaverデータ)の情報結合
df_scores = df_scores.reset_index()
#df_scores = pd.merge(df_scores, df_rankmap_data[cols_rankedmap], on=["Hash", "Difficulty"], how="left", suffixes=("", "_y"))
df_scores = pd.merge(df_scores, df_rankmap_data, on=["Hash", "Difficulty"], how="left", suffixes=("", "_y"))
df_scores = df_scores[[x for x in df_scores.columns if not x.endswith("_y")]]
df_scores = df_scores.set_index("DateJa")
# df_scores = df_scores.drop('index', axis=1).drop_duplicates()

#Score情報の保存
df_scores[(df_scores['Ranked'] == True)][cols_score].sort_index(ascending=False).to_csv(player_ranked_path)
df_scores = df_scores[(df_scores['Ranked'] == True)].sort_index(ascending=False)
df_scores.sort_index(ascending=False).to_csv(player_score_path.format(player_id))

print('ScoreCount:{}, RecentPlayCount:{}, RankedPlayCount:{}'.format(df_scores['Play'].count(), df_scores[(df_scores['Recent'] == 1)]['Play'].count(), df_scores[
                                    (df_scores['Ranked']==1)
                                    & (df_scores['Mode']=="Standard")]['Play'].count()))

# ---------------------------------------
#@title Acc再計算 
# ---------------------------------------
#@markdown ---
#@markdown <h4>Visible</h4>
recalq_visible = False #@param {type:"boolean"}

def func_max_score(x):
    combo_a = 115 * 1
    combo_b = 115 * 2
    combo_c = 115 * 4
    combo_d = 115 * 8

    if  x >= 14:
        return combo_d * (x - 13) + combo_a + combo_b * 4 + combo_c * 8
    elif x >= 6:
        return combo_c * (x - 5) + combo_a + combo_b * 4
    elif x >= 2:
        return combo_b * (x - 1) + combo_a
    elif x >= 1:
        return combo_a
    else:
        return 0

df_scores['MaxScore'] = df_scores['MaxScore']
df_scores['MaxScoreRecalq'] = df_scores['Notes'].apply(func_max_score)
df_scores['AccRecalq'] = df_scores['Score'] / df_scores['MaxScoreRecalq'] * 100
df_scores['AccRankRecalq'] = df_scores['AccRecalq'].apply(func_score)
df_scores['MaxScoreDiff'] = df_scores['MaxScore'].fillna(0) - df_scores['MaxScoreRecalq'].fillna(0)
df_scores['AccDiff'] = df_scores['Acc'].fillna(0) - df_scores['AccRecalq'].fillna(0)

df_errors = df_scores[
            (1==1)
            &(df_scores['Ranked'])
            &(df_scores['MaxScoreDiff'] != 0)
            ]

print('There are {} results where Acc is different from the game.'.format(len(df_errors)))
#print('Accがゲームと異なる結果が{}件あります。'.format(len(df_errors)))

if acc_recalq_override_is_enable:
    df_scores.rename(columns={
                            "MaxScore":"MaxScoreOrg",
                             "Acc":"AccOrg",
                             "AccRank":"AccRankOrg"
                             }, inplace=True)
    df_scores.rename(columns={
                            "MaxScoreRecalq":"MaxScore",
                             "AccRecalq":"Acc",
                             "AccRankRecalq":"AccRank"
                             }, inplace=True)
    print('Use the result of recalculating Acc.')

if recalq_visible:
    print('The following are the results extracted during recalculation.')
else:
    df_errors = df_errors.head(0)
    #print('表示設定は無効です。')

df_errors[cols_recalq].style.set_table_styles(styles_data).applymap(
    color_negative_red, subset=["AccDiff","MaxScoreDiff"]
        ).applymap(
            color_difficulty, subset=["Difficulty"]
        ).applymap(
            color_acc_rank, subset=["AccRank","AccRankRecalq"]
        # ).applymap(
        #     color_fc, subset=["FC"]
        ).format(
            style_format, na_rep="-"
        )

<IPython.core.display.Javascript object>

ScoreCount:3285, RecentPlayCount:20, RankedPlayCount:3285
There are 3 results where Acc is different from the game.
Use the result of recalculating Acc.


,Hash,Cover,SongName,Difficulty,Stars,Notes,Acc,AccRecalq,AccDiff,AccRank,AccRankRecalq,MaxScore,MaxScoreRecalq,MaxScoreDiff,Score,Miss,Combo,Preview
DateJa,,,,,,,,,,,,,,,,,,


In [4]:
#@title My Stats & Range
top_n = 100 #@param {type:"slider", min:10, max:100, step:10}
# -----------------------------
# Stats
# -----------------------------

from math import ceil
cols_temp = ["Stars","PP","Acc","Nps","Njs","Duration","Notes","Miss"]
df_scores_top_pp = df_scores[cols_temp].sort_values("PP", ascending=False).head(top_n).reset_index(drop=True)# .describe()

star_mean = df_scores_top_pp["Stars"].mean()
star_std =df_scores_top_pp["Stars"].std()
nps_mean = df_scores_top_pp["Nps"].mean()
nps_std =df_scores_top_pp["Nps"].std()
njs_mean = df_scores_top_pp["Njs"].mean()
njs_std =df_scores_top_pp["Njs"].std()
acc_mean = df_scores_top_pp["Acc"].mean()
acc_std =df_scores_top_pp["Acc"].std()
pp_mean = df_scores_top_pp["PP"].mean()
pp_std =df_scores_top_pp["PP"].std()

df_scores_top_pp.reset_index(drop=True)
df_stats = df_scores_top_pp.describe()#.drop('count', inplace=True)
df_stats.drop('count', inplace=True)
    
caption_msg = "My Top {} PPs Stats - {} - {}".format(top_n, tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])

display(df_stats.style.set_table_styles(styles_data
                                        # ).background_gradient(subset=['+PP'], cmap='coolwarm'
                                        ).format(style_format
                                        ).set_caption(caption_msg #"Playlist Simulation (now:{:,.2f}pp->target:{:,.2f}pp) - method:{} - {} - {}".format(now_pp, target_total_pp, pp_eat_target, tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])
                                        ).set_table_styles(caption_styles)
)

print("\n")

import pandas as pd

star_start = ceil((star_mean - star_std)*10)/10
star_end = round(star_mean + star_std, 1)

# DataFrameを作成
stats2 = {
        'star_start': [ceil((star_mean - star_std)*10)/10],
        'star_mean': [round(star_mean, 1)],
        'star_end': [round(star_mean + star_std, 1)],
        'nps_start': [ceil((nps_mean - nps_std)*10)/10],
        'nps_mean': [round(nps_mean, 1)],
        'nps_end': [round(nps_mean + nps_std, 1)],
        'njs_start': [ceil((njs_mean - njs_std))],
        'njs_mean': [round(njs_mean, 0)],
        'njs_end': [round(njs_mean + njs_std, 0)],
        'acc_start': [ceil((acc_mean - acc_std)*10)/10],
        'acc_mean': [round(acc_mean, 1)],
        'acc_end': [round(acc_mean + acc_std, 1)],
        'pp_start': [ceil((pp_mean - pp_std))],
        'pp_mean': [ceil(pp_mean)],
        'pp_end': [ceil(pp_mean + pp_std)],
        'Player': [df_info["name"][0]],        
         }


df_stats2 = pd.DataFrame(stats2)

def func_stats_2(x):
    star_start = x[0]
    star_end = x[1]
    nps_start = x[2]
    nps_end = x[3]
    njs_start = x[4]
    njs_end = x[5]
    return "{:,.1f}-{:,.1f}★".format(star_start, star_end),"{:,.1f}-{:,.1f}".format(nps_start, nps_end),"{:,.0f}-{:,.0f}".format(njs_start, njs_end)


df_stats2[['Stars', 'Nps', 'Njs']] = df_stats2[['star_start',
                                                'star_end',
                                                'nps_start',
                                                'nps_end',
                                                'njs_start', 
                                                'njs_end'
                                                ]].apply(func_stats_2, axis=1, result_type='expand')

caption_msg = "My Eat Range"

display(df_stats2[['Stars', 'Nps', 'Njs','Player']].style.set_table_styles(styles_data
                                        ).set_caption(caption_msg
                                        ).set_table_styles(caption_styles)
)

<IPython.core.display.Javascript object>

,Stars,PP,Acc,Nps,Njs,Duration,Notes,Miss
mean,8.45★,309.99pp,91.46%,7.72,20,209,"1,516",6
std,0.70★,8.73pp,2.89%,1.40,2,82,635,9
min,7.06★,300.21pp,81.83%,4.43,17,45,301,0
25%,7.92★,303.47pp,89.68%,6.65,19,141,"1,096",1
50%,8.40★,307.86pp,92.21%,7.51,20,209,"1,404",3
75%,8.90★,315.11pp,93.67%,8.82,21,263,"1,990",9
max,10.31★,341.14pp,95.23%,11.15,23,474,"3,451",46


,Stars,Nps,Njs,Player
0,7.8-9.2★,6.4-9.1,19-21,hatopop


# Simulation

In [12]:
#@title Curve Simulation ※

#@markdown ---
#@markdown <h2>Curve Setting ※important※</h2>

#@markdown <font size="3">This setting will be carried over to all calculations below.</font>

#@markdown <font size="3">`curve_mode`: PP curve used for the simulation</font>
curve_mode = "AGCv3" #@param ["V2.9Curve", "AGCv3"]

#@markdown <font size="3">`overwrite_pp_with_selected_curve`: Overwrite PP with the selected curve</font>

overwrite_pp_with_selected_curve  = False #@param {type:"boolean"}

#@markdown <font size="3">`c`: PP-curve parameter. Basically, don't touch it. default value:`42.113`</font>
c = 42.113 #@param {type:"number"}

#@markdown ---
#@markdown <h4>Curve Plot Setting</h4>
show_detail = False #@param {type:"boolean"}
show_plot = True #@param {type:"boolean"}
sep_x_min = 0.9 #@param {type:"number"}
sep_x_max = 1 #@param {type:"

#@markdown ---
#@title PP Curve Simulation
#@markdown <h4>Curve Sim Table Setting</h4>
show_table = True #@param {type:"boolean"}
sort_type = "PP_simu" #@param ["PP_simu","PP_org","PP_curve_diff"]
N = 25 #@param {type:"number"}


import matplotlib.pyplot as plt
import json
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px


#Vivid Color Map (鮮やかなカラーマップ)
color_map = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
    "#bcbd22", "#17becf", "#aec7e8", "#ffbb78", "#98df8a", "#ff9896", "#c5b0d5", "#c49c94",
    "#f7b6d2", "#c7c7c7", "#dbdb8d", "#9edae5"
]

#Vivid Color Map (鮮やかなカラーマップ)
color_map = [
    "#003f5c", "#58508d", "#bc5090", "#ff6361", "#ffa600", "#2f4b7c", "#665191", "#a05195",
    "#d45087", "#f95d6a", "#ff7c43", "#ffa600", "#003f5c", "#444e86", "#955196", "#dd5182",
    "#ff6e54", "#ffa600", "#7a5195", "#ef5675"
]

# Pastel Color Map (パステルカラーマップ)
color_map = [
    "#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c", "#fdbf6f", "#ff7f00",
    "#cab2d6", "#6a3d9a", "#ffff99", "#b15928", "#fbb4ae", "#b3cde3", "#ccebc5", "#decbe4",
    "#fed9a6", "#ffffcc", "#e5d8bd", "#fddaec"
]


# Plotly Default Color Map (Plotlyのデフォルトカラーマップ)
color_map = px.colors.qualitative.Plotly

# color_map = px.colors.sequential.Plotly3

# color_map = px.colors.sequential.Jet


curve_data = {
    "V2.9Curve": [
        [1, 7],
        [0.999, 6.24],
        [0.9975, 5.31],
        [0.995, 4.14],
        [0.9925, 3.31],
        [0.99, 2.73],
        [0.9875, 2.31],
        [0.985, 2.0],
        [0.9825, 1.775],
        [0.98, 1.625],
        [0.9775, 1.515],
        [0.975, 1.43],
        [0.9725, 1.36],
        [0.97, 1.3],
        [0.965, 1.195],
        [0.96, 1.115],
        [0.955, 1.05],
        [0.95, 1],
        [0.94, 0.94],
        [0.93, 0.885],
        [0.92, 0.835],
        [0.91, 0.79],
        [0.9, 0.75],
        [0.875, 0.655],
        [0.85, 0.57],
        [0.825, 0.51],
        [0.8, 0.47],
        [0.75, 0.4],
        [0.7, 0.34],
        [0.65, 0.29],
        [0.6, 0.25],
        #    [0.0, 0.0],
        [0.0001, 0.0001],#0での除算対応
    ],
    "AGCv3": [
        [0.0001, 0.0001],
        [0.6, 0.18223233667439062],
        [0.65, 0.5866010012767576],
        [0.7, 0.6125565959114954],
        [0.75, 0.6451808210101443],
        [0.8, 0.6872268862950283],
        [0.825, 0.7150465663454271],
        [0.85, 0.7462290664143185],
        [0.875, 0.7816934560296046],
        [0.9, 0.825756123560842],
        [0.91, 0.8488375988124467],
        [0.92, 0.8728710341448851],
        [0.93, 0.9039994071865736],
        [0.94, 0.9417362980580238],
        [0.95, 1.0],
        [0.955, 1.0388633331418984],
        [0.96, 1.0871883573850478],
        [0.965, 1.1552120359501035],
        [0.97, 1.2485807759957321],
        [0.9725, 1.3090333065057616],
        [0.975, 1.3807102743105126],
        [0.9775, 1.4664726399289512],
        [0.98, 1.5702410055532239],
        [0.9825, 1.697536248647543],
        [0.985, 1.8563887693647105],
        [0.9875, 2.058947159052738],
        [0.99, 2.324506282149922],
        [0.99125, 2.4902905794106913],
        [0.9925, 2.685667856592722],
        [0.99375, 2.9190155639254955],
        [0.995, 3.2022017597337955],
        [0.99625, 3.5526145337555373],
        [0.9975, 3.996793606763322],
        [0.99825, 4.325027383589547],
        [0.999, 4.715470646416203],
        [0.9995, 5.019543595874787],
        [1.0, 5.367394282890631]
    ]
}

# curve_mode = "CurrentCurve"  # ここを変更して選択したいcurve_modeを設定してください
# カーブの選択
curve = curve_data[curve_mode]
# curve 配列を降順にソート
curve.sort(reverse=True, key=lambda x: x[0])
curve_y_acc = [curve[i][0] * 100 for i in range(len(curve))]
curve_c = [curve[i][1] for i in range(len(curve))]

if show_detail:
    print(f"Selected Curve: {curve_mode}")
    print("Curve Details:")
    for i, point in enumerate(curve):
        print(f"Point {i+1}: {point}")

if show_plot:
    fig = make_subplots(rows=2, cols=1)

    # for curve_name, curve_points in curve_data.items():
    for idx, (curve_name, curve_points) in enumerate(curve_data.items()):
        x = [point[0] for point in curve_points]
        y = [point[1] for point in curve_points]

        line_color = color_map[idx]

        if curve_name == curve_mode:
            line_width = 2
            line_type = "solid"
            curve_show_name = curve_mode
        else:
            line_width = 1
            line_type = "dot"

        fig.add_trace(
            go.Scatter(x=x, y=y, mode="lines", name=curve_name,
                        line=dict(
                            dash=line_type, 
                            color=line_color, 
                            width=line_width
                        )),
                    #    line=dict(width=line_width
                    #              )),
            row=1,
            col=1,
        )

        fig.add_trace(
            go.Scatter(
                x=x, y=y, mode="lines", name=curve_name, showlegend=False, #line=dict(width=line_width)
                        line=dict(
                            dash=line_type, 
                            color=line_color, 
                            width=line_width
                        ),

            ),
            row=2,
            col=1,
        )

    fig.update_xaxes(title_text="Column 1", row=1, col=1)
    fig.update_yaxes(title_text="Column 2", row=1, col=1, range=[0, 7.0])

    # fig.update_xaxes(title_text="Column 1 (0.95 - 1.0)", row=2, col=1)
    fig.update_xaxes(title_text=f"Column 1 ({sep_x_min} - {sep_x_max})", row=2, col=1, range=[sep_x_min, sep_x_max])
    fig.update_yaxes(title_text="Column 2", row=2, col=1, range=[0, 7.0])

    fig.update_yaxes(dtick=1)

    fig.update_layout(height=800, width=800)

    fig.show()

# ---------------------------
# Star Acc → PP算出用関数
# ---------------------------
_curve_y_acc = [curve[i][0] for i in range(len(curve))]
fleg = interpolate.interp1d(_curve_y_acc, curve_c)

def func_star_acc_to_pp(x):
    star = x[0]
    target_acc = x[1]
    if target_acc > 100:
        target_acc = 100
    # target_acc の値が補間範囲内に収まるようにクリップする
    target_acc = np.clip(target_acc / 100, 0.0001, None)
    k = fleg(target_acc)
    return star * c * k

# df_scores = df_scores.copy()

PP_recal = f'PP_recalq_{curve_mode}'

df_scores[PP_recal] = df_scores[['Stars', 'Acc']].apply(func_star_acc_to_pp, axis=1)

# my_ppとpp_meanの差分を計算する
df_scores['PP_recalq_diff'] =  df_scores[PP_recal].fillna(0) - df_scores['PP'].fillna(0)

list_recal_pp = df_scores[PP_recal].to_list()
list_recal_pp.sort(reverse=True)
total_recal_pp = [list_recal_pp[i] * 0.965 ** i for i in range(len(list_recal_pp))]
t_recal_pp = sum(total_recal_pp)

pp_diff = t_recal_pp - df_info['pp'][0]

# ---------------------------
# PP Curve Simulation
# ---------------------------
# Star Acc → PP算出用関数
_curve_y_acc = [curve[i][0] for i in range(len(curve))]
fleg = interpolate.interp1d(_curve_y_acc, curve_c)

# def func_star_acc_to_pp(x):
#     star = x[0]
#     target_acc = x[1]
#     if target_acc > 100:
#         target_acc = 100
#     k = fleg(target_acc/100)
#     return star * c * k

PP_recal = f'PP_{curve_mode}'

df_scores[PP_recal] = df_scores[['Stars', 'Acc']].apply(func_star_acc_to_pp, axis=1)

if "PP_org" not in df_scores.columns:
    df_scores['PP_curve_diff'] =  df_scores[PP_recal].fillna(0) - df_scores['PP'].fillna(0)
else:
    df_scores['PP_curve_diff'] =  df_scores[PP_recal].fillna(0) - df_scores['PP_org']
    
list_recal_pp = df_scores[PP_recal].to_list()
list_recal_pp.sort(reverse=True)
total_recal_pp = [list_recal_pp[i] * 0.965 ** i for i in range(len(list_recal_pp))]
t_recal_pp = sum(total_recal_pp)

pp_diff = t_recal_pp - df_info['pp'][0]

style_pp_recalq_format = {
        "Stars": "{:,.2f}★",
        PP_recal: "{:,.1f}pp",
        "PP": "{:,.1f}pp",
        "PP_org": "{:,.1f}pp",
        "PP_mean": "{:,.1f}pp",
        "PP_std": "{:,.2f}pp",
        "PP_pred": "{:,.1f}pp",
        "PP_curve_diff": "{:+,.2f}pp",
        "Acc": "{:,.2f}%",
}

if overwrite_pp_with_selected_curve :
    # if "PP" in df_scores.columns and "PP_org" not in df_scores.columns:
    if "PP_org" not in df_scores.columns:
        df_scores.rename(columns={
                                "PP": "PP_org",
                            }, inplace=True)

    df_scores["PP"] = df_scores[PP_recal]       

    print(f"\n All PP overridden by {curve_mode}.\n\n")

    # print("{:,.03f}pp->{:,.03f}pp ({:+,.03f})".format(df_info['pp'][0],t_recal_pp, pp_diff))
    caption_msg = "PP-curve simulation : {} - {:,.01f}pp (total {:,.01f}pp {:+,.02f}) - sort: {} (overridden)".format(curve_mode, t_recal_pp,df_info['pp'][0], pp_diff,  sort_type)

    if sort_type == "PP_simu":
        sort_type = "PP"

    if show_table:
        display(df_scores[['SongName','LevelAuthor','Difficulty','Stars','Acc','PP','PP_org','PP_curve_diff']].sort_values(sort_type, ascending=False).head(N).style.set_table_styles(styles_data).applymap(
            color_negative_red, subset=['PP_curve_diff']
                # ).applymap(
                #     color_difficulty, subset=["Difficulty"]
                ).background_gradient(subset=['PP_curve_diff'], cmap='coolwarm'
                ).format(
                    style_pp_recalq_format, na_rep="-"
                ).set_caption(caption_msg #"Playlist Simulation (now:{:,.2f}pp->target:{:,.2f}pp) - method:{} - {} - {}".format(now_pp, target_total_pp, pp_eat_target, tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])
                ).set_table_styles(caption_styles)
        )

else:
    if "PP_org" in df_scores.columns:
        df_scores['PP'] = df_scores['PP_org']

    if sort_type == "PP_org":
        sort_type = "PP"

    if sort_type == "PP_simu":
        sort_type = PP_recal

    caption_msg = "Curve Simulation: {} - {:,.01f}pp (total {:,.01f}pp {:+,.02f}) - sort: {} - {} - {}".format(curve_mode, t_recal_pp, df_info['pp'][0], pp_diff,  sort_type, tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])

    if show_table:
        display(df_scores[['SongName','LevelAuthor','Difficulty','Stars','Acc',PP_recal,'PP','PP_curve_diff']].sort_values(sort_type, ascending=False).head(N).style.set_table_styles(styles_data).applymap(
            color_negative_red, subset=['PP_curve_diff']
                ).background_gradient(subset=['PP_curve_diff'], cmap='coolwarm'
                ).format(
                    style_pp_recalq_format, na_rep="-"
                ).set_caption(caption_msg #"Playlist Simulation (now:{:,.2f}pp->target:{:,.2f}pp) - method:{} - {} - {}".format(now_pp, target_total_pp, pp_eat_target, tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])
                ).set_table_styles(caption_styles)
        )


<IPython.core.display.Javascript object>

,SongName,LevelAuthor,Difficulty,Stars,Acc,PP_AGCv3,PP,PP_curve_diff
DateJa,,,,,,,,
2023-02-24 16:14:24+09:00,Blaster,That_Narwhal,ExpertPlus,8.26★,94.67%,341.1pp,341.1pp,-0.06pp
2023-05-16 18:01:37+09:00,Haunted Bus,"Dexty, Atomo, & Jabob",Expert,9.80★,89.44%,336.8pp,336.8pp,-0.01pp
2023-02-24 12:16:04+09:00,Oyasumi,Alice,ExpertPlus,8.49★,94.00%,336.7pp,336.7pp,-0.01pp
2023-05-16 19:11:53+09:00,Enigma,miitchel & Alice,ExpertPlus,9.36★,90.80%,332.8pp,332.8pp,-0.01pp
2023-05-17 22:01:57+09:00,Made of Fire,fraies & FatBeanzoop,Expert,8.47★,93.53%,329.6pp,329.6pp,+0.05pp
2022-10-14 20:15:58+09:00,Yotsuya-san ni Yoroshiku,CoolingCloset,Expert,8.88★,91.95%,326.0pp,326.0pp,-0.01pp
2023-02-24 16:23:15+09:00,Mushussu,abcbadq & Dr_Marc,Expert,8.86★,91.96%,325.3pp,325.3pp,-0.01pp
2023-02-24 17:32:15+09:00,The Beast,Ramen Noodle,Hard,8.12★,94.14%,324.8pp,324.9pp,-0.16pp
2022-10-05 22:05:39+09:00,Ruins,Rexxz,ExpertPlus,7.66★,94.93%,321.3pp,321.3pp,-0.00pp


In [10]:
#@title PP Simulation

#@markdown ※以降のシミュレーションでここでの設定は引き継がれます。
start_pp = 300 #@param {type:"integer"}
end_pp = 700 #@param {type:"integer"}
step = 20 #@param {type:"integer"}
# eat_pp = 320 #@param {type:"integer"}
target_total_pp = 10000 #@param {type:"integer"}
target_plus_pp = 100 #param {type:"integer"}
limit_eat_count = 100 #@param {type:"integer"}
# start_star = 7 #@param {type:"integer"}
# end_star = 8 #@param {type:"integer"}
my_star_range_is_auto = False #@param {type:"boolean"}

start_star = 2 #@param {type:"slider", min:0, max:15, step:0.1}
end_star = 13 #@param {type:"slider", min:0, max:15, step:0.1}
step_star = 1 #@param {type:"slider", min:0.1, max:2, step:0.1}


if my_star_range_is_auto:
    start_star = df_stats2['star_start'][0]
    end_star = df_stats2['star_end'][0]
    print('my star range is auto.:{}-{}\n'.format(start_star,end_star))


if overwrite_pp_with_selected_curve:
    now_pp = t_recal_pp
    before_type = curve_mode
else:
    now_pp = df_info['pp'][0] 
    before_type = "now"
    
target_pp_is_active = True

# total_ppがtarget_total_ppより低い場合の再設定 
if target_total_pp <= now_pp:
    print("[caution!!!]")
    print("target_total_pp was lower than the current total_pp.")
    print("target modified: {:,.1f}pp->{:,.1f}pp({:+,.0f}pp). \n\n".format(target_total_pp, now_pp + target_plus_pp,target_plus_pp))
    target_total_pp = now_pp + target_plus_pp
    target_pp_is_active = False

# total_ppがtarget_total_ppより低い場合の再設定 
if target_total_pp <= now_pp:
    print("target_total_pp was lower than the current total_pp.")
    print("target modified: {:,.1f}pp->{:,.1f}pp({:+,.0f}pp). \n\n".format(target_total_pp, now_pp + target_plus_pp,target_plus_pp))
    target_total_pp = now_pp + target_plus_pp
    target_pp_is_active = False


# 個々の+pp算出
target_pp_range_is_active = True
while True:
    dfs = []
    loop_cnt = 0
    for pp_mogmog in range(start_pp, end_pp+1, step):
        list_pp = df_scores['PP'].to_list()#
        list_pp.append(pp_mogmog)
        list_pp.sort(reverse=True)
        index_pp = list_pp.index(pp_mogmog)
        index_weight = pp_mogmog * 0.965 ** index_pp
        total_pp = [list_pp[i] * 0.965 ** i for i in range(len(list_pp))]
        t_pp = sum(total_pp) 
        new_row = {
            'no': index_pp+1, 
            'pp': pp_mogmog, 
            'weight': index_weight,
            'total_pp': t_pp, 
            '+pp':t_pp - now_pp,
            }
        # index_pp+1, pp_mogmog, index_weight, t_pp, t_pp - now_pp))
        new_df = pd.DataFrame(new_row, index=[0])
        dfs.append(new_df)
    
    if (t_pp + (t_pp - now_pp) * int(limit_eat_count/5) > target_total_pp) or loop_cnt == 10:
        break

    end_pp += 20
    start_pp += 20
    loop_cnt += 1
    target_pp_range_is_active = False

if not target_pp_range_is_active:
    print("[caution!!!]")
    print("start_pp and end_pp was lower than the current total_pp.")
    print("range modified: {:,.1f}pp-{:,.1f}pp. \n\n".format(start_pp, end_pp))


style_simulator_format = {
        "no": "{:,.0f}",
        "pp": "{:,.0f}pp",
        "weight": "({:,.2f}pp)",
        "total_pp": "{:,.2f}pp",
        "+pp": "{:+,.2f}pp",
        # "pp(n)": "{:,.2f}pp",
        "total_pp(n)": "{:,.2f}pp",
        "+pp(n)": "{:+,.2f}pp",
}

caption_styles = [dict(selector="caption",
                       props=[
                            ("text-align", "lett"),
                              ("font-size", "150%"),
                              ("border-bottom", "1px solid"),
                              ]
                       )
                  ]


# 結合
df = pd.concat(dfs, ignore_index=True)

# 合計値算出
def func_pp_mogmog(x):
    eat_pp = x[0]
    list_pp = df_scores['PP'].to_list()
    n_count = 0
    t_pp = 0
    weight = "-"

    while t_pp < target_total_pp:
        list_pp.append(eat_pp)
        list_pp.sort(reverse=True)
        total_pp = [list_pp[i] * 0.965 ** i for i in range(len(list_pp))]
        t_pp = sum(total_pp)

        n_count += 1
        if n_count >= limit_eat_count:
            break

    return "{:,.0f}pp x {}".format(eat_pp, n_count), t_pp, (t_pp - now_pp)

df[['pp(n)', 'total_pp(n)','+pp(n)']] = df[['pp','weight','total_pp']].apply(func_pp_mogmog, axis=1, result_type='expand')

# ★-Acc算出
from scipy import interpolate

kmax = curve[0][1]
kmin = curve[-1][1]

# def func_pp_acc(row, star):
#     pp = row['pp']
#     k = pp / (star * c)
#     try:
#         if (k <= kmax) and (k >= kmin):
#             return f_acc_k(k) * 100
#         else:
#             return "-"
#     except:
#         print('ex')
#         return "err"

def func_pp_acc(row, star):
    pp = row['pp']
    k = pp / (star * c)
    try:
        if (k <= kmax) and (k >= kmin):
            return f_acc_k(k) * 100
        else:
            return float('nan')
    except:
        print('ex')
        return "err"


curve_c = [curve[i][1] for i in range(len(curve))]
_curve_y_acc = [curve[i][0] for i in range(len(curve))]
f_acc_k = interpolate.interp1d(curve_c, _curve_y_acc)

# 該当列の作成
for star_i in np.arange(start_star, end_star+step_star, step_star):
    column_name = '{:,.1f}'.format(star_i)
    df[column_name] = df.apply(func_pp_acc, args=(star_i,), axis=1)

    # 動的な列名に対するフォーマットを追加
    style_simulator_format[column_name] = "{:,.2f}"


# cap_msg = "PP Simulation (now: {:,.2f}pp->target: {:,.2f}pp) - {} - {}".format(now_pp, target_total_pp,tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])

# if overwrite_pp_with_selected_curve:

cap_msg = "PP Simulation ({}: {:,.2f}pp->target:{:,.2f}pp) - {} - {}".format(before_type, now_pp, target_total_pp,tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])


df.style.highlight_null(null_color="lightgray" # ).format(style_format)
    ).format(style_simulator_format
    ).set_table_attributes("style='display:inline'"
    ).set_caption(cap_msg
        # "PP Simulation (now:{:,.2f}pp->target:{:,.2f}pp) - {} - {}".format(now_pp, target_total_pp,tz_ja.strftime("%Y.%m.%d"), df_info["name"][0])
    ).set_table_styles(caption_styles)

<IPython.core.display.Javascript object>

,no,pp,weight,total_pp,+pp,pp(n),total_pp(n),+pp(n),2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
0,103,300pp,(7.92pp),"9,068.97pp",+0.08pp,300pp x 100,"9,071.05pp",+2.16pp,99.63,99.04,98.38,97.63,96.67,95.23,92.57,88.06,82.26,75.29,66.36,64.52
1,12,320pp,(216.25pp),"9,074.54pp",+5.65pp,320pp x 100,"9,225.61pp",+156.71pp,99.69,99.15,98.55,97.88,97.07,95.98,94.14,90.80,85.96,80.32,73.17,64.97
2,2,340pp,(328.10pp),"9,091.53pp",+22.63pp,340pp x 100,"9,697.09pp",+628.19pp,99.76,99.25,98.70,98.09,97.38,96.49,95.12,92.78,88.96,84.02,78.28,71.30
3,1,360pp,(360.00pp),"9,111.49pp",+42.59pp,360pp x 41,"10,003.33pp",+934.43pp,99.81,99.34,98.82,98.27,97.63,96.85,95.81,94.14,91.25,87.18,82.26,76.47
4,1,380pp,(380.00pp),"9,131.49pp",+62.59pp,380pp x 21,"10,010.89pp",+941.99pp,99.86,99.41,98.94,98.42,97.84,97.17,96.30,95.03,92.95,89.69,85.40,80.62
5,1,400pp,(400.00pp),"9,151.49pp",+82.59pp,400pp x 15,"10,045.76pp",+976.87pp,99.91,99.48,99.04,98.55,98.03,97.42,96.67,95.67,94.14,91.61,88.06,83.75
6,1,420pp,(420.00pp),"9,171.49pp",+102.59pp,420pp x 11,"10,019.24pp",+950.35pp,99.94,99.54,99.13,98.67,98.18,97.63,96.99,96.15,94.95,93.07,90.23,86.48
7,1,440pp,(440.00pp),"9,191.49pp",+122.59pp,440pp x 9,"10,029.71pp",+960.81pp,99.98,99.60,99.20,98.78,98.32,97.81,97.24,96.53,95.56,94.14,91.91,88.75
8,1,460pp,(460.00pp),"9,211.49pp",+142.59pp,460pp x 8,"10,079.23pp","+1,010.34pp",nan,99.65,99.27,98.87,98.44,97.98,97.45,96.81,96.04,94.88,93.17,90.63
9,1,480pp,(480.00pp),"9,231.49pp",+162.59pp,480pp x 7,"10,094.25pp","+1,025.35pp",nan,99.69,99.34,98.96,98.55,98.11,97.63,97.07,96.39,95.47,94.14,92.12


In [7]:
#@title ScatterPlot (Stars - Acc/PP)

#@markdown ---
#@markdown <h4>Enable</h4>
scatterplot_is_enable = True #@param {type:"boolean"}

#@markdown ---

#@markdown <h4>Simulation Lines</h4>
simu_line_is_enable = True #@param {type:"boolean"}
simu_line_width = 0.5 #@param {type:"slider", min:0.1, max:15, step:0.1}
simu_line_type = "dot" #@param ["solid", "dot", "dash", "dashdot"]
simu_legend_is_enable = True #@param {type:"boolean"}
#@markdown ---

#@markdown <h4>Taget Lines</h4>
target_line_is_enable = True #@param {type:"boolean"}
target_line_setting_is_auto = True #@param {type:"boolean"}
target_line_width = 1 #@param {type:"slider", min:0.1, max:15, step:0.1}
target_line_type = "solid" #@param ["solid", "dot", "dash", "dashdot"]
target_legend_is_enable = True #@param {type:"boolean"}
target_pps = "330,315,300" #@param {type:"string"}
target_pp_colors = "blue,black,red" #@param {type:"string"}
target_accs = "95,94,92" #@param {type:"string"}
target_acc_colors = "blue,black,red" #@param {type:"string"}
target_stars = "8.4,7.6" #@param {type:"string"}
target_star_colors = "blue, red" #@param {type:"string"}

#@markdown ---
#@markdown <h4>Parameters</h4>

col_x = "Stars" #@param ["Stars"]
col_y = "Acc" #@param ["PP", "Acc"]
size = "Stars" #@param ["PP", "Acc", "Stars", "Rank", "Miss", "Nps", "Njs", "Notes", "Bombs", "Obstacles", "Resets", "Errors", "Warns"]
color = "Recent" #@param ["PP", "Acc", "LevelStr", "AccRank", "Stars", "Days", "Recent","Rank", "Miss", "Nps", "Njs", "Notes", "Bombs", "Obstacles", "Resets", "Errors", "Warns"]

#@markdown ---
#@markdown <h4>Filters</h4>
filtered_star_min = 0 #@param {type:"slider", min:0, max:15, step:1}
filtered_star_max = 15 #@param {type:"slider", min:0, max:15, step:1}
filtered_pp_min = 0 #@param {type:"slider", min:0, max:1000, step:10}
filtered_pp_max = 1000 #@param {type:"slider", min:0, max:1000, step:10}
filtered_acc_min = 80 #@param {type:"slider", min:0, max:100, step:1}
filtered_acc_max = 100 #@param {type:"slider", min:0, max:100, step:1}

# plotly ファイル名設定
plot_name = "ScatterPlot"
config = {
    'toImageButtonOptions': {
        'format': 'png', 
        'filename': "{}_{}_{}".format(plot_name, df_info["name"][0], tz_ja.strftime("%Y%m%d_%H%M%S")),
        'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
    }
}

# 目標線の自動設定
if target_line_setting_is_auto:
    target_pps = "{},{},{}".format(end_pp,stats2['pp_mean'][0],start_pp)
    target_pp_colors = "blue,black,red"
    target_stars = "{},{},{}".format(end_star,stats2['star_mean'][0],start_star)
    target_star_colors = "blue,black,red"
    target_accs = "{},{},{}".format(stats2['acc_end'][0],stats2['acc_mean'][0],stats2['acc_start'][0])
    target_acc_colors = "blue,black,red"
    # target_accs = "95,94,92" #@param {type:"string"}
    # target_acc_colors = "blue,black,red" #@param {type:"string"}

if scatterplot_is_enable:
    import plotly.io as pio

    def _func_Recent(x):
        # print(x)
        if  x <= Recent:
            return "recent".format(Recent)
        else:
            return "old".format(Recent)

    def func_null_exclude(x):
        if len(x[0]) == 0:
            return 0
        else:
            return x[1] 

    df_scatter = df_scores[(df_scores['Ranked']==1)
                    & (df_scores["PP"] >= filtered_pp_min)
                    & (df_scores["PP"] <= filtered_pp_max)
                    & (df_scores["Acc"] >= filtered_acc_min)
                    & (df_scores["Acc"] <= filtered_acc_max)
                    & (df_scores["Stars"] >= filtered_star_min)
                    & (df_scores["Stars"] <= filtered_star_max)]

    df_scatter['Recent'] = df_scatter['Days'].apply(_func_Recent)

    color_map = {"old": "#636EFA", "recent": "#ff4500"}

    fig2 = px.scatter(df_scatter.sort_values(color, ascending=True)
                    , x=col_x, y=col_y, color=color,
                    size=size, 
                    hover_data=["Stars","PP","Acc","AccRank","Score","Miss","Bad","SongName","SongAuthor","LevelAuthor","Difficulty"],
                    color_discrete_map=color_map,
                    width=800, height=800)



    try:
        if df_scatter[color].dtype in ['float64', 'int64']:
            simu_legend_is_enable = False
            target_legend_is_enable = False
    except:
        pass

    # x:star - y:acc
    if simu_line_is_enable and col_x == "Stars" and col_y == "Acc":
        # simu-ppの線
        for target_pp in range(start_pp, end_pp + step, step):
            target_pp = int(target_pp)
            curve_x_star = [target_pp / (curve[i][1] * c) for i in range(len(curve))]
            # x:star, y:acc
            fig2.add_scatter(x = curve_x_star, y = curve_y_acc, mode='lines', name="sim-pp-{}".format(target_pp),
                                showlegend=simu_legend_is_enable, 
                                line=dict(
                                        dash=simu_line_type, 
                                        color='black', 
                                        width=simu_line_width
                                    )
                                )

        # simu-starの線
        for target_star in np.arange(start_star, end_star+step_star, step_star):
            fig2.add_scatter(x = [target_star, target_star], y = [0, 100], mode='lines', name="sim-star-{}".format(target_star),
                                showlegend=simu_legend_is_enable, 
                                line=dict(
                                        dash=simu_line_type, 
                                        color='black', 
                                        width=simu_line_width
                                    )
                                )
            

    # x:star - y:acc
    if target_line_is_enable and col_x == "Stars" and col_y == "Acc":
        # ppの線
        target_pps = target_pps.split(",")
        target_pp_colors = target_pp_colors.split(",")
        for target_pp, target_pp_color in zip(target_pps, target_pp_colors):
            target_pp = int(target_pp)
            curve_x_star = [target_pp / (curve[i][1] * c) for i in range(len(curve))]

            # x:star, y:acc
            fig2.add_scatter(x = curve_x_star, y = curve_y_acc, mode='lines', name="pp-{}".format(target_pp),
                                showlegend=target_legend_is_enable, 
                                line=dict(
                                        dash=target_line_type, 
                                        color=target_pp_color, 
                                        width=target_line_width
                                    )
                                )
        
        # x:star, y:acc
        target_accs = target_accs.split(",")
        target_acc_colors = target_acc_colors.split(",")
        for target_acc, target_acc_color in zip(target_accs, target_acc_colors):
            target_acc = float(target_acc)
            fig2.add_scatter(x = [-1,13], y = [target_acc, target_acc], mode='lines', name="acc-{}".format(target_acc),
                                showlegend=target_legend_is_enable, 
                                line=dict(
                                        dash=target_line_type, 
                                        color=target_acc_color, 
                                        width=target_line_width
                                    )
                                )


        target_stars = target_stars.split(",")
        target_star_colors = target_star_colors.split(",")
        for target_star, target_star_color in zip(target_stars, target_star_colors):
            target_star = float(target_star)
            fig2.add_scatter(x = [target_star, target_star], y = [0, 100], mode='lines', name="star-{}".format(target_star),
                                showlegend=target_legend_is_enable, 
                                line=dict(
                                        dash=target_line_type, 
                                        color=target_star_color, 
                                        width=target_line_width
                                    )
                                )

    # x:star - y:acc
    if simu_line_is_enable and col_x == "Stars" and col_y == "PP":
        # simu-ppの線
        for target_pp in range(start_pp, end_pp + step, step):
            target_pp = int(target_pp)
            curve_x_star = [target_pp / (curve[i][1] * c) for i in range(len(curve))]
            fig2.add_scatter(x = [-1,13], y = [target_pp, target_pp], mode='lines', name="simu-pp-{}".format(target_pp),
                                showlegend=simu_legend_is_enable, 
                                line=dict(
                                        dash=simu_line_type, 
                                        color='black', 
                                        width=simu_line_width
                                    )
                                )

        #simu-starの線
        for target_star in np.arange(start_star, end_star+step_star, step_star):
            fig2.add_scatter(x = [target_star, target_star], y = [0, 1000], mode='lines', name="sim-star-{}".format(target_star),
                                showlegend=simu_legend_is_enable, 
                                line=dict(
                                        dash=simu_line_type, 
                                        color='black', 
                                        width=simu_line_width
                                    )
                                )

    # x:star - y:pp            
    if target_line_is_enable and col_x == "Stars" and col_y == "PP":
        # ppの線
        target_pps = target_pps.split(",")
        target_pp_colors = target_pp_colors.split(",")
        for target_pp, target_pp_color in zip(target_pps, target_pp_colors):

            target_pp = int(target_pp)

            # x:star, y:acc
            fig2.add_scatter(x = [-1,13], y = [target_pp, target_pp], mode='lines', name="pp-{}".format(target_pp),
                                showlegend=target_legend_is_enable, 
                                line=dict(
                                        dash=target_line_type, 
                                        color=target_pp_color, 
                                        width=target_line_width
                                    )
                                )
        
        # accの線
        from scipy import interpolate
        _curve_y_acc = [curve[i][0] for i in range(len(curve))]
        fleg = interpolate.interp1d(_curve_y_acc, curve_c)
        target_accs = target_accs.split(",")
        target_acc_colors = target_acc_colors.split(",")
        for target_acc, target_acc_color in zip(target_accs, target_acc_colors):
            target_acc = float(target_acc)

            k = fleg(target_acc/100)
            _xs = np.arange(-1, 13, 0.5)
            pp = [x * c * k for x in _xs]

            fig2.add_scatter(x = _xs, y = pp, mode='lines', name="acc-{}".format(target_acc),
                                showlegend=target_legend_is_enable, 
                                line=dict(
                                        dash=target_line_type, 
                                        color=target_acc_color, 
                                        width=target_line_width
                                    )
                                )
            
        # star
        target_stars = target_stars.split(",")
        target_star_colors = target_star_colors.split(",")
        for target_star, target_star_color in zip(target_stars, target_star_colors):
            target_star = float(target_star)
            fig2.add_scatter(x = [target_star, target_star], y = [0, 1000], mode='lines', name="star-{}".format(target_star),
                                showlegend=target_legend_is_enable, 
                                line=dict(
                                        dash=target_line_type, 
                                        color=target_star_color, 
                                        width=target_line_width
                                    )
                                )



    fig2.update_layout(title="Ranked - ScatterPlot Analysis - {} - {} - {}".format(curve_mode, tz_ja.strftime("%Y.%m.%d"), df_info["name"][0]), title_x=0.5,
                xaxis_title = "x:{} (markersize:{}, markercolor:{})".format(col_x, size, color), yaxis_title="y:{}".format(col_y)
                )
    

    diff_x = (df_scatter[col_x].max() - df_scatter[col_x].min()) / 20
    diff_y = (df_scatter[col_y].max() - df_scatter[col_y].min()) / 20

    fig2.update_layout(
                    xaxis=dict(
                        range=(df_scatter[col_x].min()-diff_x, df_scatter[col_x].max()+diff_x)
                        ),
                    yaxis=dict(
                        range=(df_scatter[col_y].min()-diff_y, df_scatter[col_y].max()+diff_y)
                        ),
    )

    fig2.show(config=config)

<IPython.core.display.Javascript object>